In [4]:
# Load packages and Files

from pyhipp.io import h5
import numpy as np
import argparse
import h5py
from pathlib import Path
import json

from pyhipp_sims.sims.sim_info import SimInfo
from scipy.fft import rfftn, irfftn

import os
import time
from multiprocessing import Pool, cpu_count
from functools import partial

print("Package loaded successfully.")

BASE_PATH = Path('/Volumes/T7/TNG100-1-Dark')
OUTPUT_PATH = Path('/Volumes/T7/halo_web')
CONFIG_FILE = OUTPUT_PATH / 'tng100-1-dark.json'
DENSITY_FILE = OUTPUT_PATH / 'density_field.hdf5'  

N_grids = 256

num_processes = 8

data = h5py.File(OUTPUT_PATH / 'density_field.hdf5', "r")
if data:
    print("Density field data file opened successfully.")
    print(data['data'][5, 5, :5])

Package loaded successfully.
Density field data file opened successfully.
[-0.92299604 -0.96168917 -0.96205057 -0.91028528 -0.93655802]


In [57]:
# calc eigenvalue
grid_size = 256
data_type = '>f4'  # big-endian float32
input_dir = OUTPUT_PATH / 'tidal_output'
file_prefix = 'tidalfield_sample'
output_filename = OUTPUT_PATH / 'tidal_output/eigenvalues.npy'

components = {
    'Txx': os.path.join(input_dir, f'{file_prefix}_0_0'),
    'Txy': os.path.join(input_dir, f'{file_prefix}_0_1'),
    'Txz': os.path.join(input_dir, f'{file_prefix}_0_2'),
    'Tyy': os.path.join(input_dir, f'{file_prefix}_1_1'),
    'Tyz': os.path.join(input_dir, f'{file_prefix}_1_2'),
    'Tzz': os.path.join(input_dir, f'{file_prefix}_2_2'),
}

tensor_parts = {}
for name, path in components.items():
    if not os.path.exists(path):
        raise FileNotFoundError(f"Error: Cannot find file: {path}")
    data = np.fromfile(path, dtype=data_type).reshape((grid_size, grid_size, grid_size))
    tensor_parts[name] = data
    print(f"Loaded {name}: shape {data.shape}, dtype {data.dtype}")

tensor_field = np.zeros((grid_size, grid_size, grid_size, 3, 3), dtype=np.float32)

tensor_field[..., 0, 0] = tensor_parts['Txx'].astype(np.float32)
tensor_field[..., 1, 1] = tensor_parts['Tyy'].astype(np.float32)
tensor_field[..., 2, 2] = tensor_parts['Tzz'].astype(np.float32)

tensor_field[..., 0, 1] = tensor_parts['Txy'].astype(np.float32)
tensor_field[..., 1, 0] = tensor_parts['Txy'].astype(np.float32)  

tensor_field[..., 0, 2] = tensor_parts['Txz'].astype(np.float32)
tensor_field[..., 2, 0] = tensor_parts['Txz'].astype(np.float32)  
tensor_field[..., 1, 2] = tensor_parts['Tyz'].astype(np.float32)
tensor_field[..., 2, 1] = tensor_parts['Tyz'].astype(np.float32)  

print("Tensor field constructed successfully.")
print(f"Tensor field shape: {tensor_field.shape}")
print(f"Tensor field dtype: {tensor_field.dtype}")

print(tensor_field[0, 0, 0])

Loaded Txx: shape (256, 256, 256), dtype >f4
Loaded Txy: shape (256, 256, 256), dtype >f4
Loaded Txz: shape (256, 256, 256), dtype >f4
Loaded Tyy: shape (256, 256, 256), dtype >f4
Loaded Tyz: shape (256, 256, 256), dtype >f4
Loaded Tzz: shape (256, 256, 256), dtype >f4
Tensor field constructed successfully.
Tensor field shape: (256, 256, 256, 3, 3)
Tensor field dtype: float32
[[ 1.0854600e+18  6.8476686e-36  4.7845632e-32]
 [ 6.8476686e-36 -1.6104735e-35 -4.0286560e-39]
 [ 4.7845632e-32 -4.0286560e-39  6.0677805e+00]]


In [6]:
density_field_filename = OUTPUT_PATH / 'density_field.hdf5'
input_dir = OUTPUT_PATH / 'tidal_output_s2.0/'
output_hdf5_file = OUTPUT_PATH / 'cf_field_sample_s2.hdf5'
n_grids = 256
l_box = 75.0 

data_type = '>f4'

eigenvalue_files = {
    'Major': 'tidalfield_sample_Major',
    'Inter': 'tidalfield_sample_Inter', 
    'Minor': 'tidalfield_sample_Minor'
}

lams = None
eigenvalue_data = {}

for name, filename in eigenvalue_files.items():
    filepath = os.path.join(input_dir, filename)
    if os.path.exists(filepath):
        data = np.fromfile(filepath, dtype=data_type).reshape((n_grids, n_grids, n_grids))
        eigenvalue_data[name] = data.astype(np.float32)
        print(f"Successfully read {name} eigenvalues: shape {data.shape}")
    else:
        print(f"Error: Cannot find eigenvalues file {filepath}")
        eigenvalue_data[name] = None

if all(v is not None for v in eigenvalue_data.values()):
    lams = np.stack([
        eigenvalue_data['Major'], 
        eigenvalue_data['Inter'],
        eigenvalue_data['Minor']   
    ], axis=-1)
    
    print(f"Successfully combined eigenvalues: shape {lams.shape}")
    print(f"Sample eigenvalues at [0,0,0]: {lams[0,0,0,:]}")
    print(f"Eigenvalue ranges:")
    print(f"  λ1 (Major): [{lams[...,0].min():.3e}, {lams[...,0].max():.3e}]")
    print(f"  λ2 (Inter): [{lams[...,1].min():.3e}, {lams[...,1].max():.3e}]") 
    print(f"  λ3 (Minor): [{lams[...,2].min():.3e}, {lams[...,2].max():.3e}]")
else:
    print("Error: Failed to read one or more eigenvalue files")
    lams = None

density_filepath = os.path.join(density_field_filename)
if os.path.exists(density_filepath):
    with h5py.File(density_filepath, 'r') as f:
        print(f.keys())
        delta_sm_x = f['data'][:]
    print(" Successfully read density field.")
else:
    print(f"Error: Cannot find density field file {density_filepath}")
    delta_sm_x = None


print(f"Creating and writing to HDF5 file: {output_hdf5_file}")

with h5py.File(output_hdf5_file, 'w') as hf:
    if delta_sm_x is not None:
        hf.create_dataset('delta_sm_x', data=delta_sm_x)
        print(" -> 'delta_sm_x'")

    if lams is not None:
        hf.create_dataset('lams', data=lams)
        print(" ->'lams'")

    hf.create_dataset('l_box', data=l_box)
    print(" ->'l_box'")
    hf.create_dataset('n_grids', data=n_grids)
    print(" ->'n_grids'")

print("\nProcessing complete.")

Successfully read Major eigenvalues: shape (256, 256, 256)
Successfully read Inter eigenvalues: shape (256, 256, 256)
Successfully read Minor eigenvalues: shape (256, 256, 256)
Successfully combined eigenvalues: shape (256, 256, 256, 3)
Sample eigenvalues at [0,0,0]: [ 3.5348145e+22 -1.7081282e-18 -1.9323100e+05]
Eigenvalue ranges:
  λ1 (Major): [nan, nan]
  λ2 (Inter): [nan, nan]
  λ3 (Minor): [nan, nan]
<KeysViewHDF5 ['data', 'l_box', 'n_grids']>
 Successfully read density field.
Creating and writing to HDF5 file: /Volumes/T7/halo_web/cf_field_sample_s2.hdf5
 -> 'delta_sm_x'
 ->'lams'
 ->'l_box'
 ->'n_grids'

Processing complete.
